## Importing required libraries:

In [ ]:
!pip install beautifulsoup4
!pip install lxml
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import random # library for random number generation

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML
from IPython.display import display_html

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
print('folium installed.')

## Data acquisition and cleaning:

In [ ]:
## Data acuisation from wikipedia page
Data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
BS=BeautifulSoup(Data,'lxml')
print(BS.title)


In [ ]:
#table = str(BS.table)
table = BS.find('table',{'class':'wikitable sortable'})
#display_html(table,raw=True)

#### Change the Html table to Panda DataFrame

In [ ]:
table_rows = table.find_all('tr')  # to read table rows

In [ ]:
dataset = []
for row in table_rows:
    dataset.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(dataset, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
print(df.head())
print(df.tail())

In [ ]:
df.info()


In [ ]:
df.shape

#### To Ignore "Not assigned" celles from "Borough" column 

In [ ]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

In [ ]:
# Reset the index
df1= df.reset_index()
df2=df1[["PostalCode","Borough","Neighbourhood"]]
df2.head()


In [ ]:
# Combining the neighbourhoods with same Postalcode
df3 = df2.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df3.reset_index(inplace=True)
df3.head(10)


In [ ]:
# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df3['Neighbourhood'] = np.where(df3['Neighbourhood'] == 'Not assigned',df3['Borough'], df3['Neighbourhood'])

df3.head()

In [ ]:
df3.shape


## Lattitud and Longtiude coordinates of Toronto's Neighborhoods

In [ ]:
#### Latitude and Longitude of Toronto's Neighborhoof is find from "http://cocl.us/Geospatial_data"
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

#### Merging df3 and lat_long tables

In [ ]:
lat_lon.rename(columns={'Postal Code':'PostalCode'}, inplace=True) #similirize the 'PostalCode' column name
df4= pd.merge(df3,lat_lon,on='PostalCode')
df4.head()

## Exploring and Clustering the Borough having the word 'Toronto'

In [ ]:
df5 = df4[df4['Borough'].str.contains('Toronto',regex=False)]
df5.reset_index(drop=True)


In [ ]:
df5.shape

#### Let's get the geographical coordinates of Toronto,Canada

In [ ]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
# finding Latitude and Longitude of Toronto, CA for Folium map
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

#### Visualizing the Dataframe of Toronto Neighborhood Borough name containe 'Toronto'

In [41]:
Toronto_map = folium.Map(location=[43.653963,-79.387207],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df5['Latitude'],df5['Longitude'],df5['Borough'],df5['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.8,
    parse_html=False).add_to(Toronto_map)
Toronto_map



## Clustering Toronto's Neighborhoods using K-mean

In [60]:
# set number of clusters
kclusters = 6
toronto_clustering = df5.drop(['PostalCode','Neighbourhood','Borough'], 1)
Kmeans = KMeans(n_clusters = kclusters,random_state=0).fit(toronto_clustering)
# check cluster labels generated for each row in the dataframe
Kmeans.labels_ 


array([0, 5, 0, 0, 4, 0, 3, 2, 0, 2, 0, 2, 4, 0, 2, 4, 0, 4, 1, 1, 1, 1,
       2, 1, 3, 2, 1, 3, 5, 1, 3, 1, 0, 3, 0, 0, 0, 3, 4], dtype=int32)

In [63]:
#insert the created cluster labels on the dataframe
df6=df5[['PostalCode','Neighbourhood','Borough','Latitude','Longitude']]
df6.insert(0,'Cluster Labels', Kmeans.labels_)


In [64]:
df6.head()

,Cluster Labels,PostalCode,Neighbourhood,Borough,Latitude,Longitude
2,0,M5A,Harbourfront,Downtown Toronto,43.654260,-79.360636
5,5,M9A,Queen's Park,Downtown Toronto,43.667856,-79.532242
9,0,M5B,"Ryerson, Garden District",Downtown Toronto,43.657162,-79.378937
15,0,M5C,St. James Town,Downtown Toronto,43.651494,-79.375418
19,4,M4E,The Beaches,East Toronto,43.676357,-79.293031


### Visualize the resulting Clusters

In [65]:
# create map
Toronto_clusters = folium.Map(location= [43.653963,-79.387207],zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df6['Latitude'], df6['Longitude'], df6['Neighbourhood'], df6['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Toronto_clusters)
       
Toronto_clusters